# Faraday Waveguide

Calculate the (circularly polarized) modes of a gyrotropic ridge waveguide.  This example incorporates imaginary off-diagonal elements in the permittivity tensor, in order to model the Faraday effect in the presence of an applied longitudinal DC magnetic field.

In [ ]:
# Enable automatic reloading of modules (IPython only)
%reload_ext autoreload
%autoreload 2 

# load required modules
import modesolver as mode
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

In [ ]:
# Refractive indices:
dn = 1.85e-4       # YIG birefringence
n1 = 1.94          # lower cladding (GGG)
n2 = 2.18          # lower cladding (Bi:YIG)
n3 = 2.19          # core (Bi:YIG)
n4 = 1             # upper cladding (air)
delta = +2.4e-4;   # Faraday rotation constant

# Vertical dimensions:
h1 = 0.5           # lower cladding
h2 = 3.1           # cladding
h3 = 3.9           # core

h4 = 0.5           # upper cladding
rh = 0.500         # Ridge height

# Horizontal dimensions
rw = 4.0           # Ridge half-width
side = 6.0         # Space on side

# Grid size:
dx = 0.100         # grid size (horizontal)
dy = 0.050         # grid size (vertical)

# Build waveguide mesh (epsxx, epsxy, epsyx, epsyy, epszz)
x,y,xc,yc,nx,ny,epsxx,edges = mode.waveguidemeshfull([n1,n2-dn,n3-dn,n4], [h1,h2,h3,h4], rh, rw, side, dx, dy, return_edges=True)
epszz = epsxx
x,y,xc,yc,nx,ny,epsyy       = mode.waveguidemeshfull([n1,n2,n3,n4], [h1,h2,h3,h4], rh, rw, side, dx, dy)
x,y,xc,yc,nx,ny,epsxy       = mode.waveguidemeshfull([0,np.sqrt(delta),np.sqrt(delta),0], [h1,h2,h3,h4], rh, rw, side, dx, dy)
epsxy = 1j*epsxy
epsyx = -epsxy

In [ ]:
# Stretch cladding mesh on all four boundaries:
x,y,xc,yc,dx,dy = mode.stretchmesh(x,y,[10,10,60,60],[2,2,5,5])

In [ ]:
wavelength = 1.485 # vacuum wavelength
nmodes = 2         # number of modes to compute

# Calculate first two modes of waveguide
neff, Hx, Hy, Hzj = mode.wgmodes(wavelength,n3,nmodes,dx,dy,epsxx=epsxx, epsxy=epsxy, epsyx = epsyx, epsyy=epsyy, epszz=epszz, boundary='0000')
print("neff =", [f"{v:.6f}" for v in neff.real])
beta = 2*np.pi * neff.real/wavelength
Lb = -np.pi/np.diff(beta)
print(f"Lb = {Lb[0]/1000:.3f} mm")

In [ ]:
# Normalize modes
I = np.abs(Hx)**2 + np.abs(Hy)**2
Imax = np.max(I, axis=(0, 1), keepdims=True)
Hx /= np.sqrt(Imax)
Hy /= np.sqrt(Imax)

# Report the transverse Stokes parameters
S0 = np.abs(Hx)**2 + np.abs(Hy)**2
S1 = np.abs(Hx)**2 - np.abs(Hy)**2
S2 = 2 * np.real(Hx * np.conj(Hy))
S3 = 2 * np.imag(Hx * np.conj(Hy))

el = np.degrees(np.arctan(S3 / np.sqrt(S1**2 + S2**2)))
az = np.degrees(np.arctan2(S2, S1))

# Locate peak:
S0_mode0 = S0[:, :, 0]          # shape (ny+1, nx+1)
jj = np.argmax(S0_mode0)
iy, ix = np.unravel_index(jj, S0_mode0.shape)  # row=y, col=x
pk = S0_mode0[iy, ix]

print(
    f"Transverse Stokes Parameters at (x,y) = "
    f"({x[ix]:.1f},{y[iy]:.1f}) um"
)

print("Mode     S0       S1       S2       S3    az(deg)  el(deg)")

nmodes = Hx.shape[2]
for ii in range(nmodes):
    print(
        f"{ii+1:2d}   "
        f"{S0[iy, ix, ii]:7.4f}  "
        f"{S1[iy, ix, ii]:+7.4f}  "
        f"{S2[iy, ix, ii]:+7.4f}  "
        f"{S3[iy, ix, ii]:+7.4f}  "
        f"{az[iy, ix, ii]:+7.2f}  "
        f"{el[iy, ix, ii]:+7.2f}"
    )

In [ ]:
# Plot Modes (Hx, Hy)
levels = np.arange(-45,0,3)
fig, axes = plt.subplots(nmodes,2, figsize=(8,8))

for m in range(nmodes):
    axHx = axes[m,0]
    axHy = axes[m,1]

    axHx.pcolormesh(x,y,abs(Hx[:,:,m]),cmap='jet',vmax=1)
    axHx.contour(x,y,20*np.log10(abs(Hx[:,:,m])),levels,colors='white',negative_linestyles='solid', linewidths=1)
    axHx.set_aspect('equal')
    axHx.axis([min(x), max(x), min(y), max(y)])
    axHx.set_xlabel('x')
    axHx.set_ylabel('y')
    axHx.set_title(f'Hx (mode {m})')
    l = LineCollection(edges, colors='black', linewidths=1)
    axHx.add_collection(l)

    axHy.pcolormesh(x,y,abs(Hy[:,:,m]),cmap='jet',vmax=1)
    axHy.contour(x,y,20*np.log10(abs(Hy[:,:,m])),levels,colors='white',negative_linestyles='solid', linewidths=1)
    axHy.set_aspect('equal')
    axHy.axis([min(x), max(x), min(y), max(y)])
    axHy.set_xlabel('x')
    axHy.set_ylabel('y')
    axHy.set_title(f'Hy (mode {m})')
    l = LineCollection(edges, colors='black', linewidths=1)
    axHy.add_collection(l)

plt.show()

In [ ]:
# Plot Modes (RCP/LCB Basis)

Hrcp = (Hx + 1j*Hy)/np.sqrt(2)
Hlcp = (Hx - 1j*Hy)/np.sqrt(2)

levels = np.arange(-45,0,3)
fig, axes = plt.subplots(nmodes,2, figsize=(8,8))

for m in range(nmodes):
    axHl = axes[m,0]
    axHr = axes[m,1]

    axHl.pcolormesh(x,y,abs(Hlcp[:,:,m]),cmap='jet',vmax=1)
    axHl.contour(x,y,20*np.log10(abs(Hlcp[:,:,m])),levels,colors='white',negative_linestyles='solid', linewidths=1)
    axHl.set_aspect('equal')
    axHl.axis([min(x), max(x), min(y), max(y)])
    axHl.set_xlabel('x')
    axHl.set_ylabel('y')
    axHl.set_title(f'Hlcp (mode {m})')
    l = LineCollection(edges, colors='black', linewidths=1)
    axHl.add_collection(l)

    axHr.pcolormesh(x,y,abs(Hrcp[:,:,m]),cmap='jet',vmax=1)
    axHr.contour(x,y,20*np.log10(abs(Hrcp[:,:,m])),levels,colors='white',negative_linestyles='solid', linewidths=1)
    axHr.set_aspect('equal')
    axHr.axis([min(x), max(x), min(y), max(y)])
    axHr.set_xlabel('x')
    axHr.set_ylabel('y')
    axHr.set_title(f'Hrcp (mode {m})')
    l = LineCollection(edges, colors='black', linewidths=1)
    axHr.add_collection(l)

plt.show()